In [17]:
import auxiliary as aux

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


valid_true, valid = aux.train_valid_split(aux.data, frac=0.0125, seed=19)

## tests

In [21]:
predicted = aux.ImputeHelper(
    aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1, num_leaves=29)),
    aux.Step(aux.simplestat, aux.subcol[1] + aux.subcol[3], imputer=SimpleImputer(strategy='mean')),
).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 55/55 [00:00<00:00, 395.40it/s]


Final validation score: 1.20748441637958
Overall final score: 0.15525455267720448


In [22]:
predicted = aux.ImputeHelper(
    aux.Step(aux.simplestat, 'all', imputer=SimpleImputer(strategy='mean')),
).run(valid, validate_on=valid_true)

Final validation: 100%|██████████| 55/55 [00:00<00:00, 431.70it/s]


Final validation score: 1.2221578718548276
Overall final score: 0.15871013212957305


In [2]:
# predicted = aux.ImputeHelper(
#     aux.Step(aux.predictor, aux.subcol[4], estimator=LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29)),
#     aux.Step(aux.mice, aux.subcol[1] + aux.subcol[3], estimator=SGDRegressor(random_state=7, max_iter=1000), epochs=20, autosplit=False),
# ).run(valid, validate_on=valid_true)

## temp

In [4]:
# predicted = aux.ImputeHelper(
#     aux.Step(aux.cosine_stats, aux.subcol[1] + aux.subcol[2], threshold=0.7, backend='loky', chunksize=50, subsample=.0001),
# ).run(valid, validate_on=valid_true)

In [ ]:
#